# Imports

In [1]:
# Data processing
import pandas as pd
# Preprocessing modules
import absenteeism_at_work_preprocessor
import students_dropout_and_academic_success_preprocessor
import loan_preprocessor
# Sci-kit learn
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
# Random numbers for random search
import random

# Global parameters

In [2]:
n_folds = 3
seed = 0
scoring = "neg_mean_absolute_error"
shuffle_train_test = True
parameters = {
    #"hidden_layer_sizes": [(10,30,10),(20,)],
    #"activation": ["tanh", "relu"],
    #"solver": ["sgd", "adam"],
    #"alpha": [0.0001, 0.05],
    #"learning_rate": ["constant","adaptive"],
}

# Common functionalities

## Scoring function for comparison table

In [3]:
def compare_networks(networks, X_test, y_test):
    results = []
    
    for network_name, network in networks.items():
        y_pred = network.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_test, y_pred, average="weighted")
        
        results.append({
            "Network": network_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        })

    return pd.DataFrame(results)

## Benchmark neural network

In [4]:
def fit_benchmark_neural_network(X_train, y_train, seed):
    return MLPClassifier(random_state=seed).fit(X_train, y_train)

## Benchmark support vector classifier

In [5]:
def fit_benchmark_support_vector_classifier(X_train, y_train, seed):
    return SVC(random_state=seed).fit(X_train, y_train)

## Custom grid search algorithm

In [6]:
def tune_custom_grid_search_neural_network(X_train, y_train, parameters, seed, scoring, folds):
    return MLPClassifier(random_state=seed).fit(X_train, y_train) # TODO add grid search algorithm

## Custom random search algorith

In [7]:


def create_network(layers, nodes, activation, learning_rate=0.01, early_stopping=True, validation_fraction=0.1, n_iter_no_change=10):
    """
    Creates an MLP network with specified layers, nodes, activation function, learning rate, and early stopping parameters.
    """
    if len(nodes) != layers:
        raise ValueError("Length of nodes list must be equal to the number of layers")

    hidden_layer_sizes = tuple(nodes)
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, 
                          max_iter=10, learning_rate_init=learning_rate, 
                          early_stopping=early_stopping, validation_fraction=validation_fraction, 
                          n_iter_no_change=n_iter_no_change)
    return model

def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    """
    Trains the MLPClassifier model and evaluates its performance on the test set.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

def random_configuration(max_layers, max_nodes):
    """
    Generates a random configuration for the neural network.
    """
    layers = random.randint(1, max_layers)
    nodes = [random.choice([10, 20, 50, 100]) for _ in range(layers)]
    activation = random.choice(['relu', 'tanh', 'logistic'])
    return layers, nodes, activation

def tune_custom_random_neural_network(X_train, y_train, X_test, y_test, parameters, seed, scoring, folds):
    """
    Tunes a custom random neural network based on specified parameters, seed, scoring method, and number of folds.
    """
    random.seed(seed)
    best_performance = None
    best_config = None
    max_layers, max_nodes = parameters['max_layers'], parameters['max_nodes']

    for _ in range(folds):
        layers, nodes, activation = random_configuration(max_layers, max_nodes)
        model = create_network(layers, nodes, activation)
        performance = train_and_evaluate(model, X_train, y_train, X_test, y_test)

        if best_performance is None or performance > best_performance:
            best_performance = performance
            best_config = (layers, nodes, activation)

    return best_config, best_performance

# Example usage:

parameters = {
    'max_layers': 5,  # Maximum number of layers
    'max_nodes': 50, # Maximum number of nodes in a layer
}

# Assuming train_data and validation_data are defined and properly preprocessed
#train_data = (X_train, y_train)
#validation_data = (X_test, y_test)

#best_config, best_performance = tune_custom_random_neural_network(X_train, y_train, X_test, y_test, 
                                                                  parameters, seed=42, scoring='accuracy', folds=100)
#print("Best configuration:", best_config)
#print("Best performance:", best_performance)


IndentationError: unexpected indent (1774162829.py, line 65)

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

def create_network(layers, nodes, activation, learning_rate=0.01, early_stopping=True, validation_fraction=0.1, n_iter_no_change=10):
    """
    Creates an MLP network with specified layers, nodes, activation function, learning rate, and early stopping parameters.
    """
    hidden_layer_sizes = tuple([nodes] * layers)
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, 
                          max_iter=100, learning_rate_init=learning_rate, 
                          early_stopping=early_stopping, validation_fraction=validation_fraction, 
                          n_iter_no_change=n_iter_no_change)
    return model

def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    """
    Trains the MLPClassifier model and evaluates its performance on the test set.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

def hill_climbing(X_train, y_train, X_test, y_test, max_layers, max_nodes):
    """
    Performs hill climbing to find a better neural network configuration.
    """
    current_layers, current_nodes, current_activation = 1, 1, "relu"
    best_performance = None
    best_config = None

    while True:
        neighbors = []

        # Generating neighbors by varying one parameter at a time
        if current_layers < max_layers:
            neighbors.append((current_layers + 1, current_nodes, current_activation))
        if current_nodes < max_nodes:
            neighbors.append((current_layers, current_nodes + 1, current_activation))
        for activation in ['relu', 'tanh', 'logistic']:
            if activation != current_activation:
                neighbors.append((current_layers, current_nodes, activation))

        # Evaluating neighbors
        best_neighbor = None
        for neighbor in neighbors:
            layers, nodes, activation = neighbor
            model = create_network(layers, nodes, activation)
            performance = train_and_evaluate(model, X_train, y_train, X_test, y_test)

            if best_performance is None or performance > best_performance:
                best_performance = performance
                best_neighbor = neighbor

        # Check if no improvement
        if best_neighbor is None:
            return create_network(current_layers, current_nodes, current_activation).fit(X_train, y_train)

        current_layers, current_nodes, current_activation = best_neighbor

# Example usage:
# Assuming X_train, y_train, X_test, y_test are defined and properly preprocessed
parameters = {'max_layers': 5, 'max_nodes': 50}
#network = hill_climbing(X_train, y_train, X_test, y_test, **parameters)
#print(network)
#print("Best configuration:", best_config)
#print("Best performance:", best_performance)


# Absenteeism at work

## Data loading

In [9]:
absenteeism_at_work = pd.read_csv("../../data/absenteeism-at-work/data.csv", delimiter=";", index_col="ID")
X = absenteeism_at_work.drop("Reason for absence", axis=1)
y = absenteeism_at_work["Reason for absence"]

## Train test split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, shuffle=shuffle_train_test)

## Preprocessing

In [11]:
X_train = absenteeism_at_work_preprocessor.preprocess(X_train)
y_train = y_train.iloc[X_train.index] # ensure that dropped rows are not in y
X_test = absenteeism_at_work_preprocessor.preprocess(X_test)
y_test = y_test.iloc[X_test.index] # ensure that dropped rows are not in y

In [19]:
folds = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
results = pd.DataFrame()

for fold, (train_idx, test_idx) in enumerate(folds.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    #absenteeism_at_work_preprocessor.preprocess(X_train)
    #absenteeism_at_work_preprocessor.preprocess(X_test)
    
    benchmark_network = fit_benchmark_neural_network(X_train, y_train, seed=seed)
    benchmark_svc = fit_benchmark_support_vector_classifier(X_train, y_train, seed=seed)
    #grid_search_tuned_network = tune_custom_grid_search_neural_network(X_train, y_train, parameters=parameters) 
    local_search_tuned_network = hill_climbing(X_train, y_train, X_test, y_test, **parameters)
    
    networks = {
        "Benchmark neural network": benchmark_network,
        "Benchmark support vector classifier": benchmark_svc,
        #"Grid search tuned neural network": grid_search_tuned_network,
        "Local search tuned neural network": local_search_tuned_network
    }
    #print(compare_networks(networks, X_test, y_test))
    results = pd.concat([results, compare_networks(networks, X_test, y_test)], axis=0)
print(results)

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/li

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserW

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserW

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserW

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserW

/home/tobsel/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/li

                               Network  Accuracy  Precision    Recall  \
0             Benchmark neural network  0.344130   0.320244  0.344130   
1  Benchmark support vector classifier  0.190283   0.036503  0.190283   
2    Local search tuned neural network  0.186235   0.035727  0.186235   
0             Benchmark neural network  0.376518   0.383180  0.376518   
1  Benchmark support vector classifier  0.214575   0.046042  0.214575   
2    Local search tuned neural network  0.214575   0.046042  0.214575   
0             Benchmark neural network  0.349593   0.277713  0.349593   
1  Benchmark support vector classifier  0.199187   0.039837  0.199187   
2    Local search tuned neural network  0.199187   0.039675  0.199187   

   F1 Score  
0  0.310278  
1  0.061256  
2  0.059952  
0  0.335800  
1  0.075816  
2  0.075816  
0  0.299865  
1  0.066396  
2  0.066171  


/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/home/tobsel/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserW

## Fit benchmark models

In [ ]:
benchmark_network = fit_benchmark_neural_network(X_train, y_train, seed=seed)
benchmark_svc = fit_benchmark_support_vector_classifier(X_train, y_train, seed=seed)

## Apply grid search algorithm

In [ ]:
    grid_search_tuned_network = tune_custom_grid_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Apply local search algorithm

In [ ]:
local_search_tuned_network = tune_custom_local_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Test all resulting networks

In [13]:
networks = {
    "Benchmark neural network": benchmark_network,
    "Benchmark support vector classifier": benchmark_svc,
    "Grid search tuned neural network": grid_search_tuned_network,
    "Local search tuned neural network": local_search_tuned_network
}
compare_networks(networks, X_test, y_test)

NameError: name 'grid_search_tuned_network' is not defined

# Students' dropout and academic success

## Data loading

In [ ]:
students_dropout_and_academic_success = pd.read_csv("../../data/predict-students-dropout-and-academic-success/data.csv", delimiter=";")
X = students_dropout_and_academic_success.drop("Target", axis=1)
y = students_dropout_and_academic_success["Target"]

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, shuffle=shuffle_train_test)

## Preprocessing

In [ ]:
X_train = students_dropout_and_academic_success_preprocessor.preprocess(X_train)
X_test = students_dropout_and_academic_success_preprocessor.preprocess(X_test)

## Fit benchmark models

In [ ]:
benchmark_network = fit_benchmark_neural_network(X_train, y_train, seed=seed)
benchmark_svc = fit_benchmark_support_vector_classifier(X_train, y_train, seed=seed)

## Apply grid search alogorithm

In [ ]:
grid_search_tuned_network = tune_custom_grid_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Apply local search algorithm

In [ ]:
local_search_tuned_network = tune_custom_local_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Test all resulting networks

In [ ]:
networks = {
    "Benchmark neural network": benchmark_network,
    "Benchmark support vector classifier": benchmark_svc,
    "Grid search tuned neural network": grid_search_tuned_network,
    "Local search tuned neural network": local_search_tuned_network
}
compare_networks(networks, X_test, y_test)

# Loan

## Data loading

In [ ]:
loan = pd.read_csv("../../data/kaggle-competitions/loan/loan-10k.lrn.csv", index_col="ID")
X = loan.drop("grade", axis=1)
y = LabelEncoder().fit_transform(loan["grade"])

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, shuffle=shuffle_train_test)

## Preprocessing

In [ ]:
X_train = loan_preprocessor.preprocess(X_train)
X_test = loan_preprocessor.preprocess(X_test)
for column in set(X_train.columns) - set(X_test.columns):
    X_test[column] = 0 # set defaults for missing one hot encoded columns
X_test = X_test[X_train.columns] # reorder columns

## Fit benchmark models

In [ ]:
benchmark_network = fit_benchmark_neural_network(X_train, y_train, seed=seed)
benchmark_svc = fit_benchmark_support_vector_classifier(X_train, y_train, seed=seed)

## Apply grid search algorithm

In [ ]:
grid_search_tuned_network = tune_custom_grid_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Apply local search algorithm

In [ ]:
local_search_tuned_network = tune_custom_local_search_neural_network(X_train, y_train, parameters=parameters, seed=seed, scoring=scoring, folds=folds)

## Test all resulting networks

In [ ]:
networks = {
    "Benchmark neural network": benchmark_network,
    "Benchmark support vector classifier": benchmark_svc,
    "Grid search tuned neural network": grid_search_tuned_network,
    "Local search tuned neural network": local_search_tuned_network
}
compare_networks(networks, X_test, y_test)